In [1]:
# 初期化
# 実行前に必ず__init__.pyを確認すること．

from __init__ import subject_number, exp, threshold_move, threshold_time, aruduino_no, columns, path_data, times_number
import sys

import serial
import keyboard
import time
import pandas as pd

sys.dont_write_bytecode = True # __pycache__を消す

df = [] # 1回分の動作データを格納
dfs = [] # dfを格納

print('subject_number is '+ str(subject_number))
print('times_number is '+ str(times_number))


subject_number is 4
times_number is 1


In [8]:
####################################
# データ収集
# エンターキーでデータ収集を停止．
# Arduino IDEが動作している場合，アクセス拒否されるので，Arduino IDEを閉じる
# エンターキーで停止しなかった場合，アクセス拒否されるので，カーネルを再起動する
####################################

ser = serial.Serial(aruduino_no, 115200, timeout=None) # arduinoからデータをとってくる

cnt = -1

while cnt <= 9:
    if keyboard.is_pressed("enter"): # エンターキーで終了
        break
    
    line = ser.readline() # ここで一行データを取得するがbyte型
    stripped_str = str(line, 'utf-8').strip() # byte型を文字列に変換して前後の空白改行除去
    stripped_str = stripped_str.split(',') # ,文字で区切る

    data = []
    for item in stripped_str: # float型に変換
        try:
            data.append(float(item))
        except ValueError:
            break
    
    df.append(data)

    if keyboard.is_pressed("space"): # 1回の動作が終了したときの処理
        df = pd.DataFrame(df, columns=columns) # 統計量を計算しやすいようにdataframeに変換
        dfs.append(df) # 動作をまとめて格納
        df = df.apply(lambda x : x.tolist(), axis=1).tolist() # appendで高速に格納するためlistに戻す
        df.clear() # 中身を空っぽに
        time.sleep(0.3) # スペースキーのチャタリングが発生するため，カウンタを遅らせる．
        cnt += 1
        print(cnt)

ser.close() # arduinoとの通信を停止

non_df = dfs.pop(0) # 最初の触れていない回を削除

0
1
2
3
4
5
6
7
8
9
10


In [9]:
# ExcelWriterを使用して，ひとつの動作をExcelに保存
df_path = '../data/move/no' + str(subject_number) + '_days' + str(times_number) + '_dfs.xlsx'
with pd.ExcelWriter(df_path) as writer:
    for i, df in enumerate(dfs):
        df.to_excel(writer, sheet_name=f'Sheet{i}', index=False)

# 何らかのフィルタリングの処理をしたい．
# だけど，コードでどうにかする方法を確認するためにいったんexcelに置き換える．
# 以下のコードで元に戻る．

dfs = pd.read_excel(df_path, sheet_name=None)
dfs = list(dfs.values())

In [10]:
####################################
# 特徴量抽出
####################################

averages = [] # 平均を格納するためのリスト
variances = [] # 分散を格納するための空のリスト
std_deviations = [] # 標準偏差を格納するための空のリスト
kurtosis_values = [] # 尖度を格納するための空のリスト
skewness_values = [] # 歪度を格納するための空のリスト
element_counts = [] # 要素数を格納するための空のリスト

for idx, df in enumerate(dfs):
    # 列ごとの特徴量を計算しリストに追加
    column_averages = df.apply(pd.to_numeric, errors='coerce', axis=0).mean(axis=0)
    averages.append(column_averages)
    column_variances = df.apply(pd.to_numeric, errors='coerce', axis=0).var(axis=0)
    variances.append(column_variances)
    column_std = df.apply(pd.to_numeric, errors='coerce', axis=0).std(axis=0)
    std_deviations.append(column_std)
    column_kurtosis = df.apply(pd.to_numeric, errors='coerce', axis=0).kurtosis(axis=0) 
    kurtosis_values.append(column_kurtosis)
    column_skewness = df.apply(pd.to_numeric, errors='coerce', axis=0).skew(axis=0)
    skewness_values.append(column_skewness)
    column_element_counts = df.apply(pd.to_numeric, errors='coerce', axis=0).count(axis=0)
    element_counts.append(column_element_counts)

# データフレームに変換    
averages_df = pd.DataFrame(averages)
variances_df = pd.DataFrame(variances)
std_deviations_df = pd.DataFrame(std_deviations)
kurtosis_df = pd.DataFrame(kurtosis_values)
skewness_df = pd.DataFrame(skewness_values)
times_df = pd.DataFrame(element_counts)

times_df = times_df[['ax']] # 1列目の値でDataFrameを分割

# ヘッダー行の文字の語尾に特徴名を追加
new_columns = [col + "_averages" for col in averages_df.columns]
averages_df.columns = new_columns
new_columns = [col + "_variances" for col in variances_df.columns]
variances_df.columns = new_columns
new_columns = [col + "_std_deviations" for col in std_deviations_df.columns]
std_deviations_df.columns = new_columns
new_columns = [col + "_kurtosis" for col in kurtosis_df.columns]
kurtosis_df.columns = new_columns
new_columns = [col + "_skewness" for col in skewness_df.columns]
skewness_df.columns = new_columns
new_columns = ["times" for col in times_df.columns]
times_df.columns = new_columns

# リストの要素数だけ1を持つDataFrameを作成
class_df = pd.DataFrame([subject_number] * len(dfs), columns=['class'])

# 各特徴が格納されたdataframeを連結
concatenated_df = class_df.join([averages_df, variances_df, std_deviations_df, kurtosis_df, skewness_df, times_df])

# とても短い動作を除去
df_filtered = concatenated_df[concatenated_df['times'] > threshold_time]

df = df_filtered
print(df.head())
df.to_csv(path_data + '/out_no'+ str(subject_number) + '_days' + str(times_number) + '.csv', mode = 'a', header=exp, index=False)

   class  ax_averages  ay_averages  az_averages  gx_averages  gy_averages  \
0      3    -0.102439     0.046829     0.980000    -0.005854     0.017073   
1      3    -0.068382     0.037353     0.986029    -0.005735     0.004853   
2      3    -0.096250     0.046000     0.986000    -0.015250    -0.009750   
3      3    -0.078333     0.040370     0.992222    -0.012963    -0.008519   
4      3    -0.113784     0.050000     0.980541    -0.010541    -0.001081   

   gz_averages  mx_averages  my_averages  mz_averages  ...  az_skewness  \
0    -0.004634     0.044390     0.430000    -0.053659  ...    -1.787744   
1    -0.000588     0.044853     0.435000    -0.064265  ...    -2.318673   
2     0.001750     0.043500     0.430750    -0.061750  ...    -1.982965   
3    -0.000556     0.044074     0.431296    -0.064259  ...    -2.238218   
4    -0.000541     0.039189     0.429459    -0.053784  ...    -1.890430   

   gx_skewness  gy_skewness  gz_skewness  mx_skewness  my_skewness  \
0     1.003879  